In [ ]:
#this generator produce a fixed scalar multiplication module with desired value and width reperesented in fixed point.
#it might use addertree for the addition part 

In [1]:
import math
import os
import sys
from FixedPoint import FXfamily, FXnum
import re
#from FixedPoint import FixedPoint

In [4]:
def Fixed_Value_Multiplier(inBias,outBias,scale,integerPartLen,fractionPartLen,IN_WIDTH,OUT_WIDTH,filesLocation,printToFile):
    
    
    IN_DATA_LENGHT=IN_WIDTH
    OUT_DATA_LENGHT=OUT_WIDTH

    #maxrangenum=pow(2,wordLen-1)-1
    #minrangenum=-pow(2,wordLen-1)
    #print('min value in selected range is: {} \nmax value in selected range is: {}'.format(minrangenum,maxrangenum))
    #print('min value in selected range is: {} \n max value in selected range is: {}'.format(FXnum(minrangenum,Q_M_N)),FXnum(maxrangenum,Q_M_N))

    Q_M_N = FXfamily(fractionPartLen,integerPartLen)
    fixedpointInBias=FXnum(inBias,Q_M_N)
    fixedpointoutBias=FXnum(outBias,Q_M_N)
    fixedPointValue=FXnum(scale,Q_M_N)
    fixedPointScale=fixedPointValue.toBinaryString()
    print("fixedpointscale binary is :"+fixedPointScale)
    print('//scale value in Q{}.{} is: {} (original vlaue: {})\nbinary form of fixedpoint scale value: {}'.format(integerPartLen,fractionPartLen,fixedPointValue,scale,fixedPointScale))
    moduleName="Fixed_{}_Multiplier_Q{}_{}".format(re.sub('\.','_',str(fixedPointValue)),integerPartLen,fractionPartLen)
    fileName=moduleName+".v"
    
    #directory and file creation operation.
    location=filesLocation
    if(printToFile):
        if(not os.path.exists(location)):
            os.mkdir(location)
        fileObject=open(location+"/"+fileName,"w+")
        #print(location+"/"+fileName) 
    else:
        fileObject=sys.stdout

    enableAdderTree=0#should set to 1 if using adder tree is needed
    numOfShiftResult=0  
    for i in range (len(fixedPointScale)):
        if(fixedPointScale[i]=='1'):
            numOfShiftResult+=1
    #numOfAddition=Rounding Up (log2(numofshiftresult))
    numOfAddition=math.ceil(math.log(numOfShiftResult,2))
   # if(numOfShiftResult>1):
   #     OUT_DATA_LENGHT+=numOfAddition# beacuse that we are doing an actual multiplication, the out put is 2*INwidth no more bits is needed.
    numOfShiftResult=0 
    #writing verilog codes in file.
    print("`timescale 1ns / 1ps",file=fileObject)
    print("//this module scales input with the desiered width by value: {}".format(fixedPointValue),file=fileObject)
    print("//the numeric representation is: Q{}.{}".format(integerPartLen,fractionPartLen),file=fileObject)
    print("//communication interface standard is AXIStream4  \n\n")
    print("module "+moduleName+" #(",file=fileObject)
    print("parameter IN_DATA_LENGHT= {}, //value is not changable but for better readability it defined as parameter.".format(IN_DATA_LENGHT),file=fileObject)
    print("parameter OUT_DATA_LENGHT= {} //value is not changable but for better readability it defined as parameter.".format(OUT_DATA_LENGHT),file=fileObject)
    print(")( \n",file=fileObject) 
    print("input aclk,".format(),file=fileObject)
    print("input aresetn,".format(),file=fileObject)
    print("input enable,".format(),file=fileObject)
    print("input [IN_DATA_LENGHT-1:0]s_axi_data,".format(),file=fileObject)
    print("input s_axi_valid,".format(),file=fileObject)
    print("input m_axi_ready,".format(),file=fileObject)
    print("output reg[OUT_DATA_LENGHT-1:0]m_axi_data,".format(),file=fileObject)
    print("output reg m_axi_valid,".format(),file=fileObject)
    print("output reg s_axi_ready".format(),file=fileObject)
    print(");\n".format(),file=fileObject)
    print("reg outready;",file=fileObject)
    print("reg [OUT_DATA_LENGHT-1:0]outdata;",file=fileObject)
    print("reg inready;",file=fileObject)
    print("reg [IN_DATA_LENGHT-1:0]indata;",file=fileObject)
    for i in range (len(fixedPointScale)):
        if(fixedPointScale[i]=='1'):
            if(i <= integerPartLen-1):
                #print('LSA by: {}'.format(integerPartLen-i-1),file=fileObject)#this line will be changed by proper code to genearte shifter in verilog code
                print("wire [{}:{}] shifted{};".format(OUT_DATA_LENGHT-1,0,numOfShiftResult),file=fileObject)
                print("assign shifted{}=s_axi_data<<<{};".format(numOfShiftResult,integerPartLen-i-1),file=fileObject)
            elif(i > integerPartLen):
                #print('RSA by: {}'.format(i-integerPartLen),file=fileObject)#this line will be changed by proper code to genearte shifter in verilog code   
                print("wire [{}:{}] shifted{};".format(OUT_DATA_LENGHT-1,0,numOfShiftResult),file=fileObject)
                print("assign shifted{}=s_axi_data>>>{};".format(numOfShiftResult,i-integerPartLen),file=fileObject)
            numOfShiftResult+=1
    print('number of shifted results is: {}'.format(numOfShiftResult))
    print('number of adder needed is: {}'.format(numOfAddition))
    if(not enableAdderTree):
        print("always@(posedge aclk)begin",file=fileObject)
        print(" if(enable)begin",file=fileObject)
        print("  if(!aresetn)begin",file=fileObject)
        print("   outdata<=0; ",file=fileObject)
        print("   outready<=0;",file=fileObject)
        print("  end",file=fileObject)
        print("  else begin",file=fileObject)
        print("   if(inready==1)begin",file=fileObject)
        print("    outdata<=", end='',file=fileObject)
        for i in range(numOfShiftResult):
            print("shifted{}".format(i),end='',file=fileObject)
            if(i!=numOfShiftResult-1):
                print("+",end='',file=fileObject)
            else:
                print(";",file=fileObject)
        print("    outready<=1;",file=fileObject)
        print("   end",file=fileObject)
        print("   else begin",file=fileObject)
        print("    outready<=0;",file=fileObject)
        print("   end",file=fileObject)
        print("  end",file=fileObject)
        print(" end",file=fileObject)
        print("end",file=fileObject)
        
        #in the following lines we will add AXIStream protocol logic blocks.
        print("/////////////////AXIStream logic/////////////".format(),file=fileObject)
        print("always @(posedge aclk)begin".format(),file=fileObject)
        print(" if(aresetn==0)begin".format(),file=fileObject)
        print("  m_axi_data<=0;".format(),file=fileObject)
        print("  m_axi_valid<=0;".format(),file=fileObject)
        print(" end".format(),file=fileObject)
        print(" else begin".format(),file=fileObject)
        print("  if(m_axi_ready==1 && m_axi_valid==1)begin".format(),file=fileObject)
        print("   m_axi_valid<=0;".format(),file=fileObject)
        print("  end".format(),file=fileObject)
        print("  else if(outready==1)begin".format(),file=fileObject)
        print("   m_axi_valid<=1;".format(),file=fileObject)
        print("   m_axi_data<=outdata;".format(),file=fileObject)
        print("  end".format(),file=fileObject)
        print(" end".format(),file=fileObject)
        print("end".format(),file=fileObject)    
        
        print("always @(posedge aclk)begin".format(),file=fileObject)
        print(" if(aresetn==0)begin".format(),file=fileObject)
        print("  s_axi_ready<=1;".format(),file=fileObject)
        print("  inready<=0;".format(),file=fileObject)
        print("  indata<=0;".format(),file=fileObject)
        print(" end".format(),file=fileObject)
        print(" else begin".format(),file=fileObject)
        print("  inready<=0;".format(),file=fileObject)
        print("  if(s_axi_valid==1 && s_axi_ready==1)begin".format(),file=fileObject)
        print("   s_axi_ready<=0;".format(),file=fileObject)
        print("   inready<=1;".format(),file=fileObject)
        print("   indata<= s_axi_data;".format(),file=fileObject)  
        print("  end".format(),file=fileObject)
        print("  else if(m_axi_valid==1 && m_axi_ready==1)begin".format(),file=fileObject)
        print("   s_axi_ready<=1;".format(),file=fileObject)
        print("  end".format(),file=fileObject)
        print(" end".format(),file=fileObject)
        print("end".format(),file=fileObject)     
    else:
        print("always@(posedge aclk)begin",file=fileObject)
        print("end",file=fileObject)
    print("endmodule".format(),file=fileObject) 
    if(printToFile):
        fileObject.close()
    return moduleName

In [6]:
scale=2.015451
fractionPartLen=10
integerPartLen=3
filesLocation="C:/Users/Delavar/OneDrive/thesis/abbasi/Scripts_/Scripts_/Generator/FixedValueMultiplierGeneratedVerilogCodes"
printToFile=0;
#Fixed_Value_Multiplier(scale,integerPartLen,fractionPartLen,filesLocation,printToFile)

//scale value in Q3.10 is: 2.014 (original vlaue: 2.015451)
binary form of fixedpoint scale value: 010.0000001111
`timescale 1ns / 1ps
//this module scales input with the desiered width by value: 2.014
//the numeric representation is: Q3.10
//communication interface standard is AXIStream4  


module Fixed_2_014_Multiplier_Q3_10 #(
parameter IN_DATA_LENGHT= 13, //value is not changable but for better readability it defined as parameter.
parameter OUT_DATA_LENGHT= 26 //value is not changable but for better readability it defined as parameter.
)( 

input aclk,
input aresetn,
input enable,
input [IN_DATA_LENGHT-1:0]s_axi_data,
input s_axi_valid,
input m_axi_ready,
output reg[OUT_DATA_LENGHT-1:0]m_axi_data,
output reg m_axi_valid,
output reg s_axi_ready
);

reg outready;
reg [OUT_DATA_LENGHT-1:0]outdata;
reg inready;
reg [IN_DATA_LENGHT-1:0]indata;
wire [25:0] shifted1;
assign shifted1=s_axi_data<<<1;
wire [25:0] shifted2;
assign shifted2=s_axi_data>>>7;
wire [25:0] shifted3;
assign shifted